# Исследование объявлений о продаже квартир

В нашем распоряжении данные сервиса Яндекс.Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Нужно научиться определять рыночную стоимость объектов недвижимости. Ваша задача — установить параметры. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта, ближайшего парка и водоёма. 

### Загрузка библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [2]:
df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t') 
pd.set_option('display.max_columns', None)
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/real_estate_data.csv'

In [ ]:
df.tail(10)

In [ ]:
df.sample(5)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

Проблемы с данными, которые необходимо проанализировать:

1) отсутствую значения, много значений, очень много значений
2) некачественные названия столбцов
3) некорректные типы данных
4) странная группировка столбцов в таблице


In [ ]:
df.hist(figsize=(15, 20))

### Предобработка данных

In [ ]:
df.columns.tolist()

In [ ]:
#переименуем столбцы в более понятные и добавим единицы измерения.
df.columns = ['total_images',
 'last_price',
 'total_area_m2',
 'first_day_exposition',
 'number_of_rooms',
 'ceiling_height_m',
 'total_floors_in_house',
 'living_area_m2',
 'floor',
 'is_apartment',
 'is_studio',
 'is_open_plan',
 'kitchen_area_m2',
 'number_of_balconies',
 'town_name',
 'nearest_airport_distance_m',
 'city_center_distance_m',
 'parks_numbers_within_3km',
 'nearest_park_distance_m',
 'ponds_number_within_3km',
 'nearest_pond_distance_m',
 'days_exposition']

In [ ]:
# Убедимся, что столбцы переименованы
df.columns.tolist()

In [ ]:
#найдем количество пустых значений
df.isnull().sum().sort_values()

вывод: часть пропусков в знчениях можно объяснить тем, что люди не заполняли данные, которые а) не знали; и б) осталяли пустым строки со значение 0 или не относящися к их объекту (0 балконов или НЕ студия или НЕ апартаменты).
Но часть данных объяснить сложно. Пустые значения в столбце города - будут мешать исследованям и показывать такие объявления тоже смысла иметь не будет. Если мы не понимаем о каком населенном пункте идет речь.
Пустые значения в колонке days_exposition это либо ошибка при выгрузке данных либо объявление не опубликовано. Данный вопрос необходимо было бы уточнить у коллег, отвечающих за выгрзку данных для исследования.                

In [ ]:
# Поменяем расположение столбцов на более привычного расположения информации: от более важной и необходимой информации к менее.
df = df[[
 'town_name',  
 'total_area_m2',
 'living_area_m2',
 'kitchen_area_m2',
 'ceiling_height_m',
 'number_of_rooms',
 'number_of_balconies',
 'floor',
 'total_floors_in_house',
 'is_apartment',
 'is_studio',
 'is_open_plan',  
 'total_images',
 'last_price',
 'first_day_exposition',
 'days_exposition',
 'city_center_distance_m',
 'nearest_airport_distance_m',
 'parks_numbers_within_3km',
 'nearest_park_distance_m',
 'ponds_number_within_3km',
 'nearest_pond_distance_m'
]]

In [ ]:
df.head()

In [ ]:
#найдем количество уникальных значений населенных пунктов
len(df['town_name'].unique())

In [ ]:
# Много много значений, посмотрим что будет, если изменить регистр букв
df['town_name'] = df['town_name'].str.lower()
len(df['town_name'].unique())

In [ ]:
# этот метод не помог. Попробуем посмотреть данные
df['town_name'].unique().tolist()


In [ ]:
# Такой список анализировать тяжеловато, но точно проблема с буквой Ё. 
# также может есть ПГТ, поселок или деревня с одним названием, а может это дубли. без уточнения этой информации с этими дублями ничего сделать не получится
# попробуем убрать буквы Ё и проверить данные
df['town_name'] = df['town_name'].str.replace('ё', 'е', regex=True)
df['town_name'] = df['town_name'].str.replace('поселок городского типа', 'поселок', regex=True)
df['town_name'] = df['town_name'].str.replace('городской поселок', 'поселок', regex=True)
df['town_name'] = df['town_name'].str.replace('поселок при железнодорожной станции', 'поселок', regex=True)
df['town_name'] = df['town_name'].str.replace('поселок станции', 'поселок', regex=True)

len(df['town_name'].unique())

In [ ]:
#убрали 46 дубля
#удалим строки где отсутсвует название города вообще
df['town_name'].isna().sum()

In [ ]:
df = df.dropna(subset=['town_name'])
df['town_name'].isna().sum()

In [ ]:
# Разберемся с колонками с булевыми значениями студии, апартаментов и открытой планировки
df['is_apartment'].unique()

In [ ]:
# Nan заменим на False, так как объекты жилового фонда
df['is_apartment'] = df['is_apartment'].fillna(0)
# Переведем булев тип к цифровому во всех столбцах подобного типа для единообразия данных
df['is_apartment'] = df['is_apartment'].map({True: 1, False: 0})
df['is_studio'] = df['is_studio'].map({True: 1, False: 0})
df['is_open_plan'] = df['is_open_plan'].map({True: 1, False: 0})

In [ ]:
# решим вопрос со пустыми значениями на балконы
df['number_of_balconies'].unique()

In [ ]:
df['number_of_balconies'] = df['number_of_balconies'].fillna(0)

In [ ]:
df.head()

### Посчитайте и добавьте в таблицу новые столбцы

In [ ]:
# Уберем метры из столбцов с прудами, аэропортами и центрами городам, приведем все к километрам
df['city_center_distance_km'] = df['city_center_distance_m'] / 1000
df['nearest_airport_distance_km'] = df['nearest_airport_distance_m'] / 1000
df['nearest_park_distance_km'] = df['nearest_park_distance_m'] / 1000
df['nearest_pond_distance_km'] = df['nearest_pond_distance_m'] / 1000
del df['city_center_distance_m']
del df['nearest_airport_distance_m']
del df['nearest_park_distance_m']
del df['nearest_pond_distance_m']
df.head()

In [ ]:
# изменим формат данных по дню публикации объявления
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format="%Y-%m-%dT%H:%M:%S")

In [ ]:
# Добавим столбцы c днем, месяцем и годом
df['day_of_week_exposition'] = df['first_day_exposition'].dt.day_name()
df['month_exposition'] = df['first_day_exposition'].dt.month_name()
df['year_exposition'] = df['first_day_exposition'].dt.year

In [ ]:
df = df[[
 'town_name',
 'total_area_m2',
 'living_area_m2',
 'kitchen_area_m2',
 'ceiling_height_m',
 'number_of_rooms',
 'number_of_balconies',
 'floor',
 'total_floors_in_house',
 'is_apartment',
 'is_studio',
 'is_open_plan',
 'total_images',
 'last_price',
 'first_day_exposition',
 'day_of_week_exposition',
 'month_exposition',
 'year_exposition',
 'days_exposition',
 'city_center_distance_km',
 'nearest_airport_distance_km',
 'parks_numbers_within_3km',
 'nearest_park_distance_km',
 'ponds_number_within_3km',
 'nearest_pond_distance_km'
]]
df.head(10)

In [ ]:
# Добавим тип этажа
def floor_status(row):
    if row['floor'] == 1:
        return 'первый'
    elif row['floor'] == row['total_floors_in_house']:
        return 'последний'
    else:
        return 'другой'
df['floor_status'] = df.apply(floor_status, axis=1)

In [ ]:
# Добавим цену за квадратный метр
df['price_per_meter'] = df['last_price'] / df['total_area_m2']
df['price_per_meter'] = df['price_per_meter'].round(decimals=2)
df.head()

In [ ]:
#добавим данные по площади комнат
df['total_living_area_ratio'] = df['living_area_m2'] / df['total_area_m2']
df['total_kitchen_area_ratio'] = df['kitchen_area_m2'] / df['total_area_m2']
df[['total_living_area_ratio', 'total_kitchen_area_ratio']].isnull().sum()

In [ ]:
df.describe()

In [ ]:
#проверим выбросы и аномальные значения

df.head()

Необычные параметры есть, довольно странными выглядят:

- максимальная площадь в 900 метров в квадрате
- высота потолка в 1 метр или 100 м
- дом в 60 этажей
- 19 комнат в квартире
также необходимо избавиться от выбросов по колонкам, критичным для исследования:
-общая площадь, цена за кв.м, срок публикации, цена реализации

In [ ]:
#исправим странные значения по высоте потолков
df['ceiling_height_m'].where(df['ceiling_height_m'] < 20, df['ceiling_height_m']/10)
df['ceiling_height_m'].hist(bins=40, range = (0,10))

In [ ]:
df.describe()

In [ ]:
df['ceiling_height_m'].median()

In [ ]:
df['ceiling_height_m'] = df['ceiling_height_m'].fillna(value= 2.65) 

In [ ]:
#напишем функцию для удалению экстримальных значений
def delete_outliers(df, column):
    q1 = df[column].quantile(0.05)                 
    q3 = df[column].quantile(0.95)
    iqr = q3 - q1
    filtered = (df[column] >= (q1 - 1.5*iqr)) & (df[column] <= (q3 + 1.5*iqr))
    return df.loc[filtered]

In [ ]:
def filtered_outliers(df, column):
    q1 = df[column].quantile(0.05)                 
    q3 = df[column].quantile(0.95)
    iqr = q3 - q1
    filtered = (df[column] >= (q1 - 1.5*iqr)) & (df[column] <= (q3 + 1.5*iqr))
    return filtered

In [ ]:
print(filtered_outliers(df,'total_area_m2'))

In [ ]:

#сравним показатели по общей площади
df.boxplot(['total_area_m2'], figsize=(10,10)).set_ylim(0,175)

In [ ]:
df.head(5)

In [ ]:
delete_outliers(df,'total_area_m2').boxplot(['total_area_m2'], figsize=(10,10)).set_ylim(0,175)

In [ ]:
df.loc[filtered_outliers(df,'total_area_m2')].shape[0]/df.shape[0]

In [ ]:
#выбрасов стало меньше. заменяем занчение в данных на отредактированные
#не работает((
df = df.loc[filtered_outliers(df,'total_area_m2')]

In [ ]:
df.boxplot(['last_price'], figsize=(10,10)).set_ylim(0,60000000)

In [ ]:
delete_outliers(df,'last_price').boxplot(['last_price'], figsize=(10,10)).set_ylim(0,60000000)

In [ ]:
df.loc[filtered_outliers(df,'last_price')].shape[0]/df.shape[0]

In [ ]:
df = df.loc[filtered_outliers(df,'last_price')]

#проверим отклонения по сроку публикации
df.boxplot(['days_exposition'], figsize=(10,10)).set_ylim(0,600)


In [ ]:
delete_outliers(df,'days_exposition').boxplot(['days_exposition'], figsize=(10,10)).set_ylim(0,600)

In [ ]:
df.loc[filtered_outliers(df,'days_exposition')].shape[0]/df.shape[0]

In [ ]:
df= df.loc[filtered_outliers(df,'days_exposition')]

In [ ]:
df.boxplot(['ceiling_height_m'], figsize=(10,10)).set_ylim(0,6)

In [ ]:
df.loc[df['ceiling_height_m'] < 10 ].boxplot(['ceiling_height_m'], figsize=(10,10)).set_ylim(0,6)

In [ ]:
df.loc[filtered_outliers(df,'ceiling_height_m')].shape[0]/df.shape[0]

In [ ]:
df= df.loc[filtered_outliers(df,'ceiling_height_m')]

In [ ]:
df.boxplot(['number_of_rooms'], figsize=(10,10)).set_ylim(0,15)

In [ ]:
delete_outliers(df,'number_of_rooms').boxplot(['number_of_rooms'], figsize=(10,10)).set_ylim(0,15)

In [ ]:
df.loc[filtered_outliers(df,'number_of_rooms')].shape[0]/df.shape[0]

In [ ]:
df= df.loc[filtered_outliers(df,'number_of_rooms')]

###  Проведение исследовательского анализа данных:

In [ ]:
df.describe()

In [ ]:
#построим гистограммы для всех параментором из задания
df['total_area_m2'].hist(bins=200, range = (0, 350))

Вывод: средняя площадь квартиры 40-50 кв, также среди популярных 6 60-65 кв.м).

In [ ]:
df['living_area_m2'].hist(bins=200, range = (0, 150))

In [ ]:
df['last_price'].unique()

In [ ]:
(df['last_price']/1000000).hist(bins=100, range = (0, 15))

большинство квартир продается в диапазоне от 2.5 до 5 млн рублей

In [ ]:
df['kitchen_area_m2'].hist(bins=50)

Площадь кухни в большинстве квартир стандартная: около 8-10 кв.м. 

In [ ]:
df['number_of_rooms'].hist(bins=100, range = (0,10))

Самые "ходовые" квартиры однокомнатные и двухкомнатные. Другие варианты значительно менее популярные.

In [ ]:
df['ceiling_height_m'].hist(bins=30, range = (0,5))

Потолки в большинстве квартир соответвуют стандарту 2.5-2.7 м

In [ ]:
df['floor'].hist(bins=100)

Квартир в пятиэтажках намного больше чем в более высотных домах.

In [ ]:
df['floor_status'].hist(bins=10)

In [ ]:
df['total_floors_in_house'].hist(bins=50)

In [ ]:
df['city_center_distance_km'].hist(bins=50)

Большинство продаваемых квартир расположены в радиусе 5-10 км от центра города

In [ ]:
df['nearest_airport_distance_km'].hist(bins=50)

In [ ]:
df['nearest_park_distance_km'].hist(bins=50)

рядом с большинством домов, в радиусе 500 м есть парк

In [ ]:
#расчитаем влияние даты публикации на итоговую цену
df['days_exposition'].hist(bins=70,range =(0,400) )

большая часть объявлений закрывается за 60 дней, чаще всего за 7 нелеь

In [ ]:
df['month_exposition'].hist(bins=30,figsize=(15,4), orientation ='vertical',  )

Большинство объялений публиковались в феврале марте, а также в осенние месяцы

In [ ]:
print(df['days_exposition'].mean())
print(df['days_exposition'].median())

In [ ]:
df.boxplot(['days_exposition'], figsize=(10,10)).set_ylim(0,600)


Вывод: Среднее время нахождения объявления о продаже объекта находится на сайте 190 дней. Половина объектов продается быстрее чем за 100 дней. Продажа быстрее чем за 50 дней можно считать быстрой, а вот продажу от полугода можно уже считать долгой.

In [ ]:
#Какие факторы больше всего влияют на общую (полную) стоимость объекта?
corr = round(df[['last_price','total_area_m2','living_area_m2','kitchen_area_m2','number_of_rooms','floor_status' ]].corr(),2)
corr.style.background_gradient(cmap='coolwarm')

Вывод: Наибольшее влияние на итоговую стоимость квартиры влияет общая площадь квартиры (0,69) и чуть меньшее влияние имеет жилая площадь. площадь кухни и общее число комнат также имеет влияние на стоимость, но связь слабая.

In [ ]:
df.pivot_table(index = 'floor_status', values = 'last_price', aggfunc='mean' ).round()

Вывод:Квартиры на первом этаже продаются дешевле всего. Они в среднем почти на миллион дешевле чем квартиры на других этажах.

In [ ]:
df.boxplot('price_per_meter', by='day_of_week_exposition',figsize=(10,10)).set_ylim(0,230000)

In [ ]:
df_1 =df.pivot_table( index= ['total_area_m2','living_area_m2','kitchen_area_m2','number_of_rooms'], values = 'last_price', aggfunc='mean')

In [ ]:
pd.plotting.scatter_matrix(df_1, figsize=(9, 9)) 

In [ ]:
df.boxplot('price_per_meter', by='month_exposition',figsize=(10,10)).set_ylim(0,200000)

In [ ]:
df.boxplot('price_per_meter', by='year_exposition',figsize=(10,10)).set_ylim(0,200000)

Вывод: Месяц или день недели публикации практически не влияют на стоимость проданного объекта. Данные по годам продажи совпадают с ожидаемыми при учете общей экономической обстановки в стране.

In [ ]:
#Посчитать  среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений.
(df['town_name'].value_counts()/ df.shape[0] * 100).head(10)


In [ ]:
# распределение между Петербургом и остальными населенными пунктами очень неравномерное. но придется работать именно с такими данными
#посчитаем топ 10 по средней цене за кв.метр
df.groupby('town_name').agg({'price_per_meter':'mean'}).sort_values(by='price_per_meter', ascending=False).head(20)


In [ ]:
def town_status(row):
    if row['town_name'] == 'санкт-петербург':
        return 'Петербург'
    else:
        return 'Окрестности Петербурга'
df['town_status'] = df.apply(town_status, axis=1)
# найдем среднюю и медианную цену за кв метр в Петербурге и городах спутниках
df.groupby('town_status').agg({'price_per_meter':'mean'}).sort_values(by='price_per_meter')


In [ ]:
df.groupby('town_status').agg({'price_per_meter':'median'}).sort_values(by='price_per_meter')


In [ ]:
dftop_10 = df.groupby('town_name').agg({'town_status':'count','price_per_meter':'mean'})
dftop_10.sort_values('price_per_meter', ascending=False).head(10)


In [ ]:
dftop_10 = dftop_10.sort_values('town_status', ascending=False).head(10)
dftop_10.sort_values('price_per_meter', ascending=True).drop('town_status', axis=1).plot(kind='barh')

Вывод: Самые дорогие квартиры в Санкт-Петербурге и Пушкине. Самые дешевые в Выборге.

In [ ]:
SPB_df = df.query('town_name == "санкт-петербург"')
SPB_df.head(5)

In [ ]:
SPB_df.info()

In [ ]:
SPB_df = SPB_df.dropna(subset=['city_center_distance_km'])

In [ ]:
SPB_df['city_center_distance_km'].unique().round(0).astype(int)

In [ ]:
SPB_df['city_center_distance_km'] = SPB_df['city_center_distance_km'].round(0).astype(int) 

In [ ]:
SPB_df['city_center_distance_km'].sort_values().unique()

In [ ]:
SPB_df.pivot_table( index = 'city_center_distance_km', values ='last_price', aggfunc= ['mean','median','count']).style.format('{:.1f}', na_rep='-')


Вывод: квартиры в самом центре (0-2 км) в цене не отличаются, потом 3км средняя цена квартиры снижается примерно на 500 т.рублей. В Петербурге рассмотояние до центра сильно влияет на продажную стоимость жилья.

### Общий вывод

Для исследования нам были даны данные сервиса Яндекс Недвижимость — архив объявлений за несколько лет о продаже квартир в Санкт-Петербурге и соседних населённых пунктах.
Наша задача — выполнить предобработку данных и изучить их, чтобы найти интересные особенности и зависимости, которые существуют на рынке недвижимости.

В ходе исследования были следущие выводы относительно рынка недвижимости:
    Средняя площадь квартиры 40-50 кв, также среди популярных площадей 60-65 кв.м.
    Площадь кухни в большинстве квартир стандартная: около 8-10 кв.м.
    Самые "ходовые" квартиры однокомнатные и двухкомнатные. Другие варианты значительно менее популярные.
    Потолки в большинстве квартир соответвуют стандарту 2.5-2.7 м
    Квартир в пятиэтажках намного больше чем в более высотных домах.
    Большинство продаваемых квартир расположены в радиусе 5-10 км от центра города, а рядом в радиусе 500 м есть парк
    Квартиры на первом этаже продаются дешевле всего. Они в среднем почти на миллион дешевле чем квартиры на других этажах.
    Среднее время нахождения объявления о продаже объекта находится на сайте 190 дней. Половина объектов продается быстрее чем за 100 дней. Продажа быстрее чем за 50 дней можно считать быстрой, а вот продажу от полугода можно уже считать долгой.
    Наибольшее влияние на итоговую стоимость квартиры влияет общая площадь квартиры (0,69) и чуть меньшее влияние имеет жилая площадь. площадь кухни и общее число комнат также имеет влияние на стоимость, но связь слабая.
    Месяц или день недели публикации практически не влияют на стоимость проданного объекта. Данные по годам продажи совпадают с ожидаемыми при учете общей экономической обстановки в стране.
    Самые дорогие квартиры в Санкт-Петербурге и Пушкине. Самые дешевые в Выборге.
    Квартиры в самом центре (0-2 км) в цене не отличаются, потом 3км средняя цена квартиры снижается примерно на 500 тыс.рублей. В Петербурге расстояние до центра сильно влияет на продажную стоимость жилья.
